<a href="https://colab.research.google.com/github/tortoisehare/TSR-GAN/blob/master/TSR_CIFAR_Classifier_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Stephanie Tietz

LeNet5 model design and grayscale preprocessing idea from: mohamedameen93 on full German Traffic Sign dataset

https://github.com/mohamedameen93/German-Traffic-Sign-Classification-Using-TensorFlow

Regularization options provided, Adadelta or Adam optimizer, trained on German TS Dataset plus "not a sign" images, optimized for 3 classes first then expanding to 44

Multiclass Image Classification in TensorFlow

In [0]:
import numpy as np
np.random.seed(1187) #to help reproduce

from __future__ import print_function
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
#import random
#import cv2
#from skimage.filters import rank
#from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras import optimizers
#from keras.utils import np_utils
#from keras import backend as K


Using TensorFlow backend.


In [0]:
#upload data
#from google.colab import files

#uploaded = files.upload()

#traindata = np.load('smalltrain.npy') #only 3 classes, both circle signs
traindata = np.load('alltrain.npy') #all classes

#testdata = np.load('smalltest.npy')
testdata = np.load('alltest.npy')

validdata = np.load('allvalid.npy')


In [0]:
'''
X_train, Y_train = tstrain['features'], tstrain['labels']
X_valid, Y_valid = tsvalid['features'], tsvalid['labels']
X_test, Y_test = tstest['features'], tstest['labels']

num_train = X_train.shape[0]
num_test = X_test.shape[0]
num_valid = X_valid.shape[0]
'''


In [0]:
#traindata = np.load('smalltrain2.npy') #smalltrain2 has triangle sign
#testdata = np.load('smalltest2.npy')

#traindata = np.load('tsrtrain.npy')
#testdata = np.load('tsrtest.npy')

print(traindata.shape) #(34799,3073) for signs
print(testdata.shape) #(12630,3073) for signs
print(validdata.shape) #(4410,3073) for signs
print(testdata)

In [0]:
#signs = []
#with open('signnames.csv', 'r') as csvfile:
    #signnames = csv.reader(csvfile, delimiter=',')
    #next(signnames,None)
    #for row in signnames:
     #   signs.append(row[1])
    #csvfile.close()

In [0]:

#split into x and y
X_train = traindata[:,:-1]
Y_train = traindata[:,-1]
#Y_train = Y_train.reshape(len(Y_train),1)
print(X_train.shape)
print(Y_train.shape)

X_test = testdata[:,:-1]
Y_test = testdata[:,-1]
#Y_test = Y_test.reshape(len(Y_test),1)
print(X_test.shape)
print(Y_test.shape)

X_valid = validdata[:,:-1]
Y_valid = validdata[:,-1]
#Y_valid = Y_valid.reshape(len(Y_valid),1)
print(X_valid.shape)
print(Y_valid.shape)

#reshape into (32,32,3) for each example for this model input
X_train = (X_train.reshape(traindata.shape[0], 3, 32, 32)).transpose([0,2,3,1])
print(X_train.shape)
X_test = (X_test.reshape(testdata.shape[0], 3, 32, 32)).transpose([0,2,3,1])
print(X_test.shape)
X_valid = (X_valid.reshape(validdata.shape[0], 3, 32, 32)).transpose([0,2,3,1])
print(X_valid.shape)


(34799, 3072)
(34799,)
(12630, 3072)
(12630,)
(4410, 3072)
(4410,)
(34799, 32, 32, 3)
(12630, 32, 32, 3)
(4410, 32, 32, 3)


In [0]:
#START OF DATA PREPROCESSING
#Shuffle

from sklearn.utils import shuffle

#Randomize (shuffle) the data
print(Y_train)
X_train, Y_train = shuffle(X_train, Y_train)
X_valid, Y_valid = shuffle(X_valid, Y_valid)
X_test, Y_test = shuffle(X_test, Y_test)
print(Y_train)



[41 41 41 ... 25 25 25]
[25  3  2 ... 13 18 34]


In [0]:
#Make grayscale
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)
grayscale = [0.299,0.587,0.144]

X_test = np.dot(X_test, grayscale)
X_train = np.dot(X_train, grayscale)
X_valid = np.dot(X_valid, grayscale)
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)




(34799, 32, 32, 3)
(12630, 32, 32, 3)
(4410, 32, 32, 3)
(34799, 32, 32)
(12630, 32, 32)
(4410, 32, 32)


In [0]:
#Normalize data
X_train = np.array(X_train)/255
X_test = np.array(X_test)/255
X_valid = np.array(X_valid)/255
print(X_train.shape)

(34799, 32, 32)


In [0]:
#Expand dimensions to fit 4D input array

X_train = np.expand_dims(X_train,-1)
print(X_train.shape)

X_test = np.expand_dims(X_test,-1)
print(X_test.shape)

X_valid = np.expand_dims(X_valid,-1)
print(X_valid.shape)

(34799, 32, 32, 1)
(12630, 32, 32, 1)
(4410, 32, 32, 1)


In [0]:
#compute mean image of training set and subtract from training images (not test images)
#chann_index_swap = np.swapaxes(X_train,0,3)
#mean_image = [[[sum(pixel)/len(pixel) for pixel in col] for col in row] for row in chann_index_swap]
#mean_image = np.swapaxes(mean_image,0,2)
#mean_image = np.swapaxes(mean_image,0,1)

#X_train = X_train - mean_image
#print(X_train.shape)

In [0]:
num_classes = 44 #change to 44 if including cifar

assert(len(X_train)==len(Y_train))
n_train = len(X_train)
assert(len(X_test)==len(Y_test))
n_test = len(X_test)
assert(len(X_valid)==len(Y_valid))
n_valid = len(X_valid)

#Useful image variables
im_rows = 32
im_cols = 32
image_shape = (32, 32, 1)


In [0]:
#Histogram visualization of classes
hist_data = np.histogram(Y_train, bins=range(num_classes+1))
hist_map = {}
for occr,i in zip(hist_data[0], hist_data[1]):
  hist_map[occr] = i

plt.hist(Y_train, bins=range(num_classes+1), color = '#00ffCC')
plt.axis([0,num_classes,0,2500])
plt.show()
print("Train Class with most examples:")
print(hist_map[np.amax(hist_data[0])],np.amax(hist_data[0]))
print("Class with least examples:")
print(hist_map[np.amin(hist_data[0])],np.amin(hist_data[0]))

hist_data = np.histogram(Y_valid, bins=range(num_classes+1))
hist_map = {}
for occr,i in zip(hist_data[0], hist_data[1]):
  hist_map[occr] = i

plt.hist(Y_valid, bins=range(num_classes+1), color = '#00ffCC')
plt.axis([0,num_classes,0,250])
plt.show()
print("Valid Class with most examples:")
print(hist_map[np.amax(hist_data[0])],np.amax(hist_data[0]))
print("Class with least examples:")
print(hist_map[np.amin(hist_data[0])],np.amin(hist_data[0]))

hist_data = np.histogram(Y_test, bins=range(num_classes+1))
hist_map = {}
for occr,i in zip(hist_data[0], hist_data[1]):
  hist_map[occr] = i

plt.hist(Y_test, bins=range(num_classes+1), color = '#00ffCC')
plt.axis([0,num_classes,0,1000])
plt.show()
print("Test Class with most examples:")
print(hist_map[np.amax(hist_data[0])],np.amax(hist_data[0]))
print("Class with least examples:")
print(hist_map[np.amin(hist_data[0])],np.amin(hist_data[0]))


In [0]:
class LeNet:  

    def __init__(self, n_out=44, mu=0, sigma=0.1, learning_rate=0.001):
        # Hyperparameters
        self.mu = mu
        self.sigma = sigma

        # Layer 1 (Convolutional): Input = 32x32x1. Output = 28x28x6.
        self.filter1_width = 5
        self.filter1_height = 5
        self.input1_channels = 1
        self.conv1_output = 6
        # Weight and bias
        self.conv1_weight = tf.Variable(tf.truncated_normal(
            shape=(self.filter1_width, self.filter1_height, self.input1_channels, self.conv1_output),
            mean = self.mu, stddev = self.sigma))
        self.conv1_bias = tf.Variable(tf.zeros(self.conv1_output))
        # Apply Convolution
        self.conv1 = tf.nn.conv2d(x, self.conv1_weight, strides=[1, 1, 1, 1], padding='VALID') + self.conv1_bias
        
        # Activation:
        self.conv1 = tf.nn.relu(self.conv1)
        
        # Pooling: Input = 28x28x6. Output = 14x14x6.
        self.conv1 = tf.nn.max_pool(self.conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        
        # Layer 2 (Convolutional): Output = 10x10x16.
        self.filter2_width = 5
        self.filter2_height = 5
        self.input2_channels = 6
        self.conv2_output = 16
        # Weight and bias
        self.conv2_weight = tf.Variable(tf.truncated_normal(
            shape=(self.filter2_width, self.filter2_height, self.input2_channels, self.conv2_output),
            mean = self.mu, stddev = self.sigma))
        self.conv2_bias = tf.Variable(tf.zeros(self.conv2_output))
        # Apply Convolution
        self.conv2 = tf.nn.conv2d(self.conv1, self.conv2_weight, strides=[1, 1, 1, 1], padding='VALID') + self.conv2_bias
        
        # Activation:
        self.conv2 = tf.nn.relu(self.conv2)
        
        # Pooling: Input = 10x10x16. Output = 5x5x16.
        self.conv2 = tf.nn.max_pool(self.conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        
        # Flattening: Input = 5x5x16. Output = 400.
        self.fully_connected0 = Flatten()(self.conv2)
        
        # Layer 3 (Fully Connected): Input = 400. Output = 120.
        self.connected1_weights = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = self.mu, stddev = self.sigma))
        self.connected1_bias = tf.Variable(tf.zeros(120))
        self.fully_connected1 = tf.add((tf.matmul(self.fully_connected0, self.connected1_weights)), self.connected1_bias)
        
        # Activation:
        self.fully_connected1 = tf.nn.relu(self.fully_connected1)
    
        # Layer 4 (Fully Connected): Input = 120. Output = 84.
        self.connected2_weights = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = self.mu, stddev = self.sigma))
        self.connected2_bias = tf.Variable(tf.zeros(84))
        self.fully_connected2 = tf.add((tf.matmul(self.fully_connected1, self.connected2_weights)), self.connected2_bias)
        
        # Activation.
        self.fully_connected2 = tf.nn.relu(self.fully_connected2)
    
        # Layer 5 (Fully Connected): Input = 84. Output = 44.
        self.output_weights = tf.Variable(tf.truncated_normal(shape=(84, 44), mean = self.mu, stddev = self.sigma))
        self.output_bias = tf.Variable(tf.zeros(44))
        self.logits =  tf.add((tf.matmul(self.fully_connected2, self.output_weights)), self.output_bias)

        
        # Training operation
        self.one_hot_y = tf.one_hot(y, n_out)
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.one_hot_y)
        self.loss_operation = tf.reduce_mean(self.cross_entropy)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        self.training_operation = self.optimizer.minimize(self.loss_operation)

        # Accuracy operation

        self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.one_hot_y,1))
        self.accuracy_operation = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))


        # Saving all variables
        self.saver = tf.train.Saver()
    
    def y_predict(self, X_data, BATCH_SIZE=64):
        num_examples = len(X_data)
        y_pred = np.zeros(num_examples, dtype=np.int32)
        sess = tf.get_default_session()
        for offset in range(0, num_examples, BATCH_SIZE):
            #print("in y_pred offset")
            batch_x = X_data[offset:offset+BATCH_SIZE]
            y_pred[offset:offset+BATCH_SIZE] = sess.run(tf.argmax(self.logits, 1), feed_dict={x:batch_x, keep_prob:1, keep_prob_conv:1})
            print(y_pred)
        return y_pred
    
    def evaluate(self, X_data, y_data, BATCH_SIZE=64):
        num_examples = len(X_data)
        total_accuracy = 0
        sess = tf.get_default_session()
        for offset in range(0, num_examples, BATCH_SIZE):
            batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
            #print("in eval offset")
            accuracy = sess.run(self.accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0, keep_prob_conv: 1.0 })
            #print("after acc in offset")
            total_accuracy += (accuracy * len(batch_x))
        return total_accuracy / num_examples

In [0]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))

keep_prob = tf.placeholder(tf.float32)       # For fully-connected layers
keep_prob_conv = tf.placeholder(tf.float32)  # For convolutional layers

EPOCHS = 30
BATCH_SIZE = 64
DIR = 'Saved_Models'

In [0]:
LeNet_Model = LeNet(n_out = num_classes)
model_name = "LeNet"

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(Y_train)
    print("Training ...")
    print()
    for i in range(EPOCHS):
        #X_train, Y_train = shuffle(X_train, Y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], Y_train[offset:end]
            sess.run(LeNet_Model.training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob : 0.5, keep_prob_conv: 0.7})
            
        validation_accuracy = LeNet_Model.evaluate(X_valid, Y_valid)
        print("EPOCH {} : Validation Accuracy = {:.3f}%".format(i+1, (validation_accuracy*100)))
    LeNet_Model.saver.save(sess, os.path.join(DIR, model_name))
    print("Model saved")

Instructions for updating:
Colocations handled automatically by placer.
Training ...

EPOCH 1 : Validation Accuracy = 6.984%
EPOCH 2 : Validation Accuracy = 9.932%
EPOCH 3 : Validation Accuracy = 10.340%
EPOCH 4 : Validation Accuracy = 9.909%
EPOCH 5 : Validation Accuracy = 10.204%
EPOCH 6 : Validation Accuracy = 10.658%
EPOCH 7 : Validation Accuracy = 10.181%
EPOCH 8 : Validation Accuracy = 9.909%
EPOCH 9 : Validation Accuracy = 9.660%
EPOCH 10 : Validation Accuracy = 9.637%
EPOCH 11 : Validation Accuracy = 9.320%
EPOCH 12 : Validation Accuracy = 9.637%
EPOCH 13 : Validation Accuracy = 9.524%
EPOCH 14 : Validation Accuracy = 9.184%
EPOCH 15 : Validation Accuracy = 9.002%
EPOCH 16 : Validation Accuracy = 8.866%
EPOCH 17 : Validation Accuracy = 8.322%
EPOCH 18 : Validation Accuracy = 8.231%
EPOCH 19 : Validation Accuracy = 7.982%
EPOCH 20 : Validation Accuracy = 7.937%
EPOCH 21 : Validation Accuracy = 7.823%
EPOCH 22 : Validation Accuracy = 7.937%
EPOCH 23 : Validation Accuracy = 7.732%

In [0]:

with tf.Session() as sess:
    LeNet_Model.saver.restore(sess, os.path.join(DIR, "LeNet"))
    y_pred = LeNet_Model.y_predict(X_test)
    test_accuracy = sum(Y_test == y_pred)/len(Y_test)
    print("Test Accuracy = {:.1f}%".format(test_accuracy*100))
    

cm = confusion_matrix(Y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm = np.log(.0001 + cm)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Log of normalized Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [0]:
'''
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
'''
